<img src=./imgs/model_io.jpg width=35% />

[langchain documents](https://python.langchain.com/docs/modules/model_io/models/chat/llm_chain)

[LangChain-Tutorials](https://github.com/sugarforever/LangChain-Tutorials)

In [1]:
import os

In [2]:
os.environ['OPENAI_API_KEY'] = '******'

# 自定义模版

## 需求

> 给定一个函数的英文名字， 让LLM生成英文解释<br>


为了完成这个任务：
> 1. 输入为function name的提示模版。<br>
> 2. format 提示模版，根据提供的 "source code of the function"

为什么需要自定义模版
> lanchain提供了很多模版，可以用来完成大量任务。<br>
> 但是默认模版可能不符合你的需求.<br>
> 例如：你想要一个动态指令生成的提示模版。**模版是动态的？**<br>
> 之前提到过的`PromptTemplate`,`ChatPromptTemplate` 都是默认模版

> 1. step 1: 根据函数名获取源码

In [7]:
import inspect

In [8]:
def get_source_code(function_name):
    # get the source code of the function
    return inspect.getsource(function_name)

> 2. 生成自定义模版，以function_name 为输入。 格式化提示模版以提供函数的源码。

In [9]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

In [10]:
PROMPT = """\
Given the function name and source code, generate an English language explanation of the function.
Function Name: {function_name}
Source Code:
{source_code}
Explanation:
"""

In [11]:
class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function."""
    
    @validator("input_variables")
    def validate_input_variable(cls, v):
        """validate that the input variable are correct"""
        if len(v) != 1 or 'function_name' not in v:
            raise ValueError('function_name must be the only input variable.')
        return v
    
    def format(self, **kwargs):
        # get the source code of the function
        source_code = get_source_code(kwargs['function_name'])
        
        # Generate the prompt to be sent to the language model
        prompt = PROMPT.format(function_name=kwargs['function_name'].__name__, source_code=source_code)
        return prompt
                             
    def _prompt_type(self):
        return 'function-explainer'

> 3. 使用自定义模版

In [12]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=['function_name'])

In [13]:
# generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)

In [15]:
print(prompt)

Given the function name and source code, generate an English language explanation of the function.
Function Name: get_source_code
Source Code:
def get_source_code(function_name):
    # get the source code of the function
    return inspect.getsource(function_name)

Explanation:

